In [23]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
from pickle import dump, load
import os

import sys
sys.path.append("..")

from dyntapy.supply_data import get_toy_network, relabel_graph
from dyntapy.demand_data import add_centroids, auto_configured_centroids, od_graph_from_matrix
from dyntapy.assignments import StaticAssignment
from dyntapy.visualization import show_network, show_demand, show_link_od_flows
from dyntapy.results import get_selected_link_analysis, get_od_flows

import numpy as np

# first we download a road network graph using OSMNX, and clean up some of the data. We rename some attributes
# from OpenStreetMap to comply with GMNS (https://github.com/zephyr-data-specs/GMNS) and 
# provide a very rough estimate of capacities.

g = get_toy_network('simple_merge')

In [24]:
centroid_x = np.array([2, 2, 6])
centroid_y = np.array([1, 2, 1.5])
g = add_centroids(g, centroid_x, centroid_y, euclidean=True)
# also adds connectors automatically
g = relabel_graph(g)  # adding link and node ids, connectors and centroids
# are the first elements
show_network(g, euclidean=True)
od_matrix = np.zeros(9).reshape((3, 3))
od_matrix[0, 2] = 500
od_matrix[1, 2] = 500
od_graph = od_graph_from_matrix(od_matrix, centroid_x, centroid_y)
show_demand(od_graph, euclidean=True)

In [26]:
tolls = np.empty(g.number_of_edges(), dtype=np.float64)

assignment = StaticAssignment(g,od_graph, tolls) # TODO fix tolls
methods = ['dial_b']
for method in methods:
    result = assignment.run(method)
    show_network(g, flows = result.flows, notebook=True, show_nodes=False, euclidean=True)
    print(f'{method=} ran successfully')
# all static assignments return a result object that follows the same structure, see
# below
print(result.__dict__.keys())

TypeError: __init__() takes 3 positional arguments but 4 were given